### Introduction

OpenVoice is a versatile instant voice tone transferring and generating speech in various languages with just a brief audio snippet from the source speaker. OpenVoice has three main features: (i) high quality tone color replication with multiple languages and accents; (ii) it provides fine-tuned control over voice styles, including emotions, accents, as well as other parameters such as rhythm, pauses, and intonation. (iii) OpenVoice achieves zero-shot cross-lingual voice cloning, eliminating the need for the generated speech and the reference speech to be part of a massive-speaker multilingual training dataset.

![image](https://github.com/openvinotoolkit/openvino_notebooks/assets/5703039/ca7eab80-148d-45b0-84e8-a5a279846b51)

More details about model can be found in [project web page](https://research.myshell.ai/open-voice), [paper](https://arxiv.org/abs/2312.01479), and official [repository](https://github.com/myshell-ai/OpenVoice)

This notebooks provides example of converting original OpenVoice model (https://github.com/myshell-ai/OpenVoice) to OpenVINO IR format for faster inference.

In this tutorial we will explore how to convert and run OpenVoice using OpenVINO.
#### Table of contents:
- [Clone repository and install requirements](#Clone-repository-and-install-requirements)
- [Download checkpoints and load PyTorch model](#Download-checkpoints-and-load-PyTorch-model)
- [Convert Models to OpenVINO IR](#Convert-Models-to-OpenVINO-IR)
- [Inference](#Inference)
    - [Select inference device](#Select-inference-device)
    - [Select reference tone](#Select-reference-tone)
    - [Run inference](#Run-inference)
- [Run OpenVoice Gradio online app](#Run-OpenVoice-Gradio-online-app)
- [Cleanup](#Cleanup)

## Clone repository and install requirements
[back to top ⬆️](#Table-of-contents:)

In [ ]:
import sys
from pathlib import Path

repo_dir = Path("OpenVoice")

if not repo_dir.exists():
    !git clone https://github.com/myshell-ai/OpenVoice

# append to sys.path so that modules from the repo could be imported
sys.path.append(str(repo_dir))

%pip install \
"librosa>=0.8.1" \
"wavmark>=0.0.3" \
"faster-whisper>=0.9.0" \
"pydub>=0.25.1" \
"whisper-timestamped>=1.14.2" \
"tqdm" \
"inflect>=7.0.0" \
"unidecode>=1.3.7" \
"eng_to_ipa>=0.0.2" \
"pypinyin>=0.50.0" \
"cn2an>=0.5.22" \
"jieba>=0.42.1" \
"langid>=1.1.6" \
"gradio>=4.15" \
"ipywebrtc"

In [ ]:
%pip install ffmpeg-downloader
!ffdl install -y

## Download checkpoints and load PyTorch model
[back to top ⬆️](#Table-of-contents:)

In [ ]:
import os
import torch
import openvino as ov
import ipywidgets as widgets
from IPython.display import Audio

core = ov.Core()

from api import BaseSpeakerTTS, ToneColorConverter, OpenVoiceBaseClass
import se_extractor


# Fetch `notebook_utils` module
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)
from notebook_utils import download_file

In [4]:
base_url = 'https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/'

CKPT_BASE_PATH = 'checkpoints/'

en_suffix = 'base_speakers/EN'
zh_suffix = 'base_speakers/ZH'
converter_suffix = 'converter'
en_ckpt_path = f'{CKPT_BASE_PATH}/{en_suffix}'
zh_ckpt_path = f'{CKPT_BASE_PATH}/{zh_suffix}'
converter_path = f'{CKPT_BASE_PATH}/{converter_suffix}'

To make notebook lightweight by default model for Chinese speech is not activated, in order turn on please set flag `enable_chinese_lang` to True

In [5]:
enable_chinese_lang = False

In [ ]:
download_file(base_url + f'{converter_suffix}/checkpoint.pth', directory=converter_path)
download_file(base_url + f'{converter_suffix}/config.json', directory=converter_path)
download_file(base_url + f'{en_suffix}/checkpoint.pth', directory=en_ckpt_path)
download_file(base_url + f'{en_suffix}/config.json', directory=en_ckpt_path)

download_file(base_url + f'{en_suffix}/en_default_se.pth', directory=en_ckpt_path)
download_file(base_url + f'{en_suffix}/en_style_se.pth', directory=en_ckpt_path)

if enable_chinese_lang:
    download_file(base_url + f'{zh_suffix}/checkpoint.pth', directory=zh_ckpt_path)
    download_file(base_url + f'{zh_suffix}/config.json', directory=zh_ckpt_path)
    download_file(base_url + f'{zh_suffix}/zh_default_se.pth', directory=zh_ckpt_path)

In [ ]:
pt_device = "cpu"  # todo: check if torch.device("cuda" if torch.cuda.is_available() else "cpu") is indeed needed

en_base_speaker_tts = BaseSpeakerTTS(f'{en_ckpt_path}/config.json', device=pt_device)
en_base_speaker_tts.load_ckpt(f'{en_ckpt_path}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{converter_path}/config.json', device=pt_device)
tone_color_converter.load_ckpt(f'{converter_path}/checkpoint.pth')

if enable_chinese_lang:
    zh_base_speaker_tts = BaseSpeakerTTS(f'{zh_ckpt_path}/config.json', device=pt_device)
    zh_base_speaker_tts.load_ckpt(f'{zh_ckpt_path}/checkpoint.pth')
else:
    zh_base_speaker_tts = None

## Convert models to OpenVINO IR
[back to top ⬆️](#Table-of-contents:)

There are 2 models in OpenVoice: first one is responsible for speach generation `BaseSpeakerTTS` and the second one `ToneColorConverter` imposes arbitrary voice tone to the original speech. To convert to OpenVino IR format first we need to get acceptable `torch.nn.Module` object. Both ToneColorConverter, BaseSpeakerTTS instead of using `self.forward` as the main entry point use custom `infer` and `convert_voice` methods respectively, therefore need to wrap them with a custom class that is inherited from torch.nn.Module. 

<!---
# One more reason to make a wrapper is also that these functions use float arguments while only torch.Tensor and tuple of torch.Tensors are acceptable 
# todo: check if it works when kwargs are moved to example inputs.
-->

In [8]:
voice_convert_kwargs = dict(tau=0.3)
tts_kwargs = dict(noise_scale=0.667, noise_scale_w=0.6, length_scale=1.0)  # length_scale = 1.0 / speed - here we set speed = 1 as default

class OVOpenVoiceBase(torch.nn.Module):
    """
    Base class for both TTS and voice tone conversion model: constructor is same for both of them.
    """
    def __init__(self, voice_model: OpenVoiceBaseClass, kwargs):
        super().__init__()
        self.voice_model = voice_model
        self.default_kwargs = kwargs
        for par in voice_model.model.parameters():
            par.requires_grad = False
    
class OVOpenVoiceTTS(OVOpenVoiceBase):
    """
    Constructor of this class accepts BaseSpeakerTTS object for speach generation and wraps it's 'infer' method with forward.
    """
    def get_example_input(self):
        stn_tst = self.voice_model.get_text('this is original text', self.voice_model.hps, False)
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        speaker_id = torch.LongTensor([1])
        return (x_tst, x_tst_lengths, speaker_id)

    def forward(self, x, x_lengths, sid):
        return self.voice_model.model.infer(x, x_lengths, sid, **self.default_kwargs)
    
class OVOpenVoiceConverter(OVOpenVoiceBase):
    """
    Constructor of this class accepts ToneColorConverter object for voice tone conversion and wraps it's 'voice_conversion' method with forward.
    """
    def get_example_input(self):
        y = torch.randn([1, 513, 238], dtype=torch.float32)
        y_lengths = torch.LongTensor([y.size(-1)])
        target_se = torch.randn(*(1, 256, 1))
        source_se = torch.randn(*(1, 256, 1))
        return (y, y_lengths, source_se, target_se)
    
    def forward(self, y, y_lengths, sid_src, sid_tgt):
        return self.voice_model.model.voice_conversion(y, y_lengths, sid_src, sid_tgt, **self.default_kwargs)

Convert to OpenVino IR and save to IRs_path folder for the future use. If IRs already exist skip conversion and read them directly

In [9]:
IRS_PATH = 'openvino_irs/'
EN_TTS_IR = f'{IRS_PATH}/openvoice_en_tts.xml'
ZH_TTS_IR = f'{IRS_PATH}/openvoice_zh_tts.xml'
VOICE_CONVERTER_IR = f'{IRS_PATH}/openvoice_tone_conversion.xml'

paths = [EN_TTS_IR, VOICE_CONVERTER_IR]
models = [OVOpenVoiceTTS(en_base_speaker_tts, tts_kwargs), OVOpenVoiceConverter(tone_color_converter, voice_convert_kwargs)]
if enable_chinese_lang:
    models.append(OVOpenVoiceTTS(zh_base_speaker_tts, tts_kwargs))
    paths.append(ZH_TTS_IR)
ov_models = []

for model, path in zip(models, paths):
    if not os.path.exists(path):
        ov_model = ov.convert_model(model, example_input=model.get_example_input())
        ov.save_model(ov_model, path)
    else:
        ov_model = core.read_model(path)
    ov_models.append(ov_model)

ov_en_tts, ov_voice_conversion = ov_models[:2]
if enable_chinese_lang:
    ov_zh_tts = ov_models[-1]

## Inference

### Select inference device
[back to top ⬆️](#Table-of-contents:)

In [10]:
core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)
device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

### Select reference tone
[back to top ⬆️](#Table-of-contents:)

First of all, select the reference tone of voice to which the generated text will be converted: your can select from existing ones, record your own by selecting `record_manually` or upload you own file by `load_manually`

In [11]:
REFERENCE_VOICES_PATH = f'{repo_dir}/resources/'
reference_speakers = [
    *[path for path in os.listdir(REFERENCE_VOICES_PATH)  if os.path.splitext(path)[-1] == '.mp3'],
    'record_manually',
    'load_manually',
]

ref_speaker = widgets.Dropdown(
    options=reference_speakers,
    value=reference_speakers[0],
    description="reference voice from which tone color will be copied",
    disabled=False,
)

ref_speaker

Dropdown(description='reference voice from which tone color will be copied', options=('demo_speaker0.mp3', 'de…

In [12]:
OUTPUT_DIR = 'outputs/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [13]:
ref_speaker_path = f'{REFERENCE_VOICES_PATH}/{ref_speaker.value}'
allowed_audio_types = '.mp4,.mp3,.wav,.wma,.aac,.m4a,.m4b,.webm'

if ref_speaker.value == 'record_manually':
    ref_speaker_path = f'{OUTPUT_DIR}/custom_example_sample.webm'
    from ipywebrtc import AudioRecorder, CameraStream
    camera = CameraStream(constraints={'audio': True,'video':False})
    recorder = AudioRecorder(stream=camera, filename=ref_speaker_path, autosave=True)
    display(recorder)
elif ref_speaker.value == 'load_manually':
    upload_ref = widgets.FileUpload(accept=allowed_audio_types, multiple=False, description='Select audio with reference voice')
    display(upload_ref)

Play the reference voice sample before cloning it's tone to another speech

In [14]:
def save_audio(voice_source: widgets.FileUpload, out_path: str):
    with open(out_path, 'wb') as output_file: 
        for uploaded_filename in voice_source.value:
            output_file.write(voice_source.value[0]['content'])

if ref_speaker.value == 'load_manually':
    ref_speaker_path = f'{OUTPUT_DIR}/{upload_ref.value[0].name}'
    save_audio(upload_ref, ref_speaker_path)

In [15]:
Audio(ref_speaker_path)

Load speaker embeddings

In [16]:
# ffmpeg is neeeded to load mp3 and manually recorded webm files
import ffmpeg_downloader as ffdl
delimiter = ':' if sys.platform != 'win32' else ';'
os.environ['PATH'] = os.environ['PATH'] + f"{delimiter}{ffdl.ffmpeg_dir}"

In [17]:
en_source_default_se = torch.load(f'{en_ckpt_path}/en_default_se.pth')
en_source_style_se = torch.load(f'{en_ckpt_path}/en_style_se.pth')
zh_source_se = torch.load(f'{zh_ckpt_path}/zh_default_se.pth') if enable_chinese_lang else None

target_se, audio_name = se_extractor.get_se(ref_speaker_path, tone_color_converter, target_dir='processed', vad=True)  # ffmpeg must be installed

Replace original infer methods of `OpenVoiceBaseClass` with optimized OpenVINO inference.

There are pre and post processings that are not traceable and could not be offloaded to OpenVINO, instead of writing such processing ourselves we will rely on the already existing ones. We just replace infer and voice conversion functions of `OpenVoiceBaseClass` so that the the most computationally expensive part is done in OpenVINO.

In [18]:
def assert_kwargs_are_same(kwargs: dict, orig_kwargs: dict):
    for k, v in kwargs.items():
        assert v == orig_kwargs[k], f"Model was converted to IR with {k}: '{orig_kwargs[k]}', " \
            f"but you are trying to infer with {k} = '{v}'. " \
            f"Please use original value or rerun ov.convert_model with the desirable value of '{k}'"

def get_pathched_infer(ov_model: ov.Model, device: str, orig_kwargs: dict = None) -> callable:
    compiled_model = core.compile_model(ov_model, device)
    
    def infer_impl(x, x_lengths, sid=None, **kwargs):
        assert_kwargs_are_same(kwargs, orig_kwargs)
        ov_output = compiled_model((x, x_lengths, sid))
        return (torch.tensor(ov_output[0]), )
    return infer_impl

def get_patched_voice_conversion(ov_model: ov.Model, device: str, orig_kwargs: dict = None) -> callable:
    compiled_model = core.compile_model(ov_model, device)

    def voice_conversion_impl(y, y_lengths, sid_src, sid_tgt, **kwargs):
        assert_kwargs_are_same(kwargs, orig_kwargs)
        ov_output = compiled_model((y, y_lengths, sid_src, sid_tgt))
        return (torch.tensor(ov_output[0]), )
    return voice_conversion_impl


en_base_speaker_tts.model.infer = get_pathched_infer(ov_en_tts, device.value, orig_kwargs=tts_kwargs)
tone_color_converter.model.voice_conversion = get_patched_voice_conversion(ov_voice_conversion, device.value, orig_kwargs=voice_convert_kwargs)
if enable_chinese_lang:
    zh_base_speaker_tts.model.infer = get_pathched_infer(ov_zh_tts, device.value, orig_kwargs=tts_kwargs)

### Run inference
[back to top ⬆️](#Table-of-contents:)

In [19]:
voice_source = widgets.Dropdown(
    options=['use TTS', 'choose_manually'],
    value='use TTS',
    description="Voice source",
    disabled=False,
)

voice_source

Dropdown(description='Voice source', options=('use TTS', 'choose_manually'), value='use TTS')

In [20]:
if voice_source.value == 'choose_manually':
    upload_orig_voice = widgets.FileUpload(accept=allowed_audio_types, multiple=False, 
                                description='audo whose tone will be replaced')
    display(upload_orig_voice)

In [ ]:
if voice_source.value == 'choose_manually':
    orig_voice_path = f'{OUTPUT_DIR}/{upload_orig_voice.value[0].name}'
    save_audio(upload_orig_voice, orig_voice_path)
else:
    text = """
    OpenVINO toolkit is a comprehensive toolkit for quickly developing applications and solutions that solve 
    a variety of tasks including emulation of human vision, automatic speech recognition, natural language processing, 
    recommendation systems, and many others.
    """
    orig_voice_path = f'{OUTPUT_DIR}/tmp.wav'
    en_base_speaker_tts.tts(text, orig_voice_path, speaker='default', language='English')

And finally, run voice tone conversion with OpenVINO optimized model

In [ ]:
resulting_voice_path = f'{OUTPUT_DIR}/output_with_cloned_voice_tone.wav'

tone_color_converter.convert(
    audio_src_path=orig_voice_path, 
    src_se=en_source_default_se, 
    tgt_se=target_se, 
    output_path=resulting_voice_path, 
    message="@MyShell")

In [23]:
Audio(orig_voice_path)

In [24]:
Audio(resulting_voice_path)

## Run OpenVoice Gradio online app
[back to top ⬆️](#Table-of-contents:)

We can also use [Gradio](https://www.gradio.app/) app to run TTS and voice tone conversion online.

In [ ]:
from openvoice_gradio import get_demo

demo = get_demo(OUTPUT_DIR, tone_color_converter, en_base_speaker_tts, zh_base_speaker_tts, en_source_default_se, en_source_style_se, zh_source_se)
demo.queue(max_size=2)

try:
    demo.launch(debug=True, height=1000)
except Exception:
    demo.launch(share=True, debug=True, height=1000)
# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/

## Cleanup
[back to top ⬆️](#Table-of-contents:)

In [ ]:
# please run this cell for stopping gradio interface
demo.close()

# clean up 
# import shutil
# shutil.rmtree(CKPT_BASE_PATH)
# shutil.rmtree(IRS_PATH)
# shutil.rmtree(OUTPUT_DIR)